# Vulnerability Ireland 

Population density is an important indicator to consider in a vulnerability assessment. The population density can be derived for each of the 'Small Area' units of the 2022 Census.

This script gets the population in each 'Small Area' in 2022 and the area of the 'Small Area' (in km<sup>2</sup>). These are then used to calculate the population density. 

## Setup

Load the R libraries:

In [1]:
# load the libraries
library(tidyverse)
library(sf)
library(raster)
library(dplyr)
library(rgdal)


── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Linking to GEOS 3.12.1, GDAL 3.8.3, PROJ 9.3.1; sf_use_s2() is TRUE

Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:dplyr’:

    select


Please note that rgdal will be retired during October 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.
See h

Set a temporary and export directory:

In [2]:
#set a directory for export 
exportDirectory <- "../1_InputData/1b_PopulationDensity"

## 2.0 Population Data

The population data is derived from the 2016 census which can be imported directly from the CSO website (this is the default) or using a local version:

In [3]:
#get the data from the CSO website
#smallAreaCSOData <- read.csv("https://www.cso.ie/en/media/csoie/census/census2022/SAPS_2022_Small_Area_270923.csv",  header=TRUE, sep=",")

#get the data locally
smallAreaCSOData <- read.csv('/Users/declan/Projects/REACHOUT/walther/ISVEHI-main_2024/1_InputData/CSOData/SAPS_2022_Small_Area_270923.csv', header=TRUE, sep=",", stringsAsFactors = FALSE)
colnames(smallAreaCSOData)[colnames(smallAreaCSOData) == "GUID"] ="SA_GUID__1"

#get unique ID and population data only
smallAreaPopulation <- smallAreaCSOData[, c('SA_GUID__1', 'T1_1AGETT'), drop = FALSE]
names(smallAreaPopulation)[2] <- 'populationTotal'

head(smallAreaPopulation)

,SA_GUID__1,populationTotal
,<chr>,<int>
1,00b00ae4-229d-455d-84f1-d6face4876b1,376
2,03003797-1fcd-4fcf-8dde-b2188e3fb1db,310
3,06650182-eeaa-4c6c-847c-f85ddaf5361b,375
4,08e82f06-46ee-4141-aa07-79a793a12b27,225
5,0920215b-86d3-4a53-9fc0-6008ae5c91f9,344
6,0daa02c9-4e30-45fe-90d7-6a1d3e7af2cb,164


## 3.0 'Small Areas' Spatial Data

The spatial data for the Small Areas used within the 2016 census can be imported directly from the CSO website (this is the default) or using a local version. The area of each Small Area has already been calculated and is available in the 'Shape_Are' attribute of the data - this data is converted from m<sup>2</sup> to km<sup>2</sup>. 

In [4]:
#download the small area shapefile
#download.file("http://data-osi.opendata.arcgis.com/datasets/70a33cbb8bd7406da0d571be28624721_0-.zip", 
#              destfile = "./downloads/CSO_Small_Areas_-_National_Statistical_Boundaries_-_2022_-_Ungeneralised.zip" , mode='wb')
unzip("./downloads/CSO_Small_Areas_-_National_Statistical_Boundaries_-_2022_-_Ungeneralised.zip", exdir = "./downloads")


#read the data - Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2022
smallAreaShp <- st_read('./downloads/SMALL_AREA_2022.shp')

#convert to irenet95 / irish transverse mercator
projIreNet95 <- "+proj=tmerc +lat_0=53.5 +lon_0=-8 +k=0.99982 +x_0=600000 +y_0=750000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"
smallAreaShp <- st_transform(smallAreaShp, projIreNet95)


#convert the area from m2 to km2
smallAreaShp$SHAPE_Area <- smallAreaShp$SHAPE_Area/1000000


#get the the unique IDs data and the shape areas only
smallAreaShp <- as.data.frame(smallAreaShp[, c('SA_GUID__1', 'SA_PUB2022', 'SHAPE_Area'), drop = TRUE])

head(smallAreaShp)


Reading layer `SMALL_AREA_2022' from data source 
  `/Users/declan/Projects/REACHOUT/walther/ISVEHI-main_2024/0_Scripts/downloads/SMALL_AREA_2022.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 18919 features and 28 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 417471.5 ymin: 519663.7 xmax: 734481.1 ymax: 966896.3
Projected CRS: IRENET95 / Irish Transverse Mercator


,SA_GUID__1,SA_PUB2022,SHAPE_Area
,<chr>,<chr>,<dbl>
1,4c07d11e-0a4f-851d-e053-ca3ca8c0ca7f,017010016,0.2049442
2,4c07d11e-0a3a-851d-e053-ca3ca8c0ca7f,017010046,0.2882254
3,4c07d11e-0a4e-851d-e053-ca3ca8c0ca7f,017010037,0.2084269
4,4c07d11e-0a25-851d-e053-ca3ca8c0ca7f,017010005,0.3487986
5,4c07d11e-0a57-851d-e053-ca3ca8c0ca7f,017010036,0.4293491
6,4c07d11e-2df2-851d-e053-ca3ca8c0ca7f,017010009,0.0592992


## 4.0 Combining the data

The population data needs to be joined with the area data in order to calculate the density. This is done by matching the commmon filed of the GUID to merge to two datasets together.

In [5]:
smallAreaShpPop <-  merge(smallAreaShp, smallAreaPopulation, by= "SA_GUID__1")
head(smallAreaShpPop)

,SA_GUID__1,SA_PUB2022,SHAPE_Area,populationTotal
,<chr>,<chr>,<dbl>,<int>
1,000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,087055007/01,0.07844937,260
2,00275248-3d78-4fdb-8e5c-8ff1f76aebea,187017004/01,0.19468908,183
3,002b83e4-5cce-414a-8570-871a50ae0964,107029004/01,0.19134093,263
4,005eec3f-d746-480f-ae2c-bc6f1151548d,267081006/02,0.05736744,241
5,00b00ae4-229d-455d-84f1-d6face4876b1,147002002/02,0.11281036,376
6,00df2b78-66c8-4009-89e0-0e14e15e4208,087067015/03,0.19108284,297


## 5.0 Population Density

The population density can then be calculated and stored as a new field within the dataset. The final data is then exported to a csv for use in later analysis.

In [6]:
#Population Density Calculation
smallAreaShpPop$popDensity <- smallAreaShpPop$populationTotal/smallAreaShpPop$SHAPE_Area
head(smallAreaShpPop)

,SA_GUID__1,SA_PUB2022,SHAPE_Area,populationTotal,popDensity
,<chr>,<chr>,<dbl>,<int>,<dbl>
1,000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,087055007/01,0.07844937,260,3314.2393
2,00275248-3d78-4fdb-8e5c-8ff1f76aebea,187017004/01,0.19468908,183,939.9603
3,002b83e4-5cce-414a-8570-871a50ae0964,107029004/01,0.19134093,263,1374.5099
4,005eec3f-d746-480f-ae2c-bc6f1151548d,267081006/02,0.05736744,241,4200.9895
5,00b00ae4-229d-455d-84f1-d6face4876b1,147002002/02,0.11281036,376,3333.0272
6,00df2b78-66c8-4009-89e0-0e14e15e4208,087067015/03,0.19108284,297,1554.2997


In [7]:
#Scaled Population Density
smallAreaShpPop$popDensityZ <- scale(smallAreaShpPop$popDensity)
head(smallAreaShpPop)

,SA_GUID__1,SA_PUB2022,SHAPE_Area,populationTotal,popDensity,popDensityZ
,<chr>,<chr>,<dbl>,<int>,<dbl>,"<dbl[,1]>"
1,000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,087055007/01,0.07844937,260,3314.2393,-0.08544822
2,00275248-3d78-4fdb-8e5c-8ff1f76aebea,187017004/01,0.19468908,183,939.9603,-0.48468431
3,002b83e4-5cce-414a-8570-871a50ae0964,107029004/01,0.19134093,263,1374.5099,-0.41161460
4,005eec3f-d746-480f-ae2c-bc6f1151548d,267081006/02,0.05736744,241,4200.9895,0.06365922
5,00b00ae4-229d-455d-84f1-d6face4876b1,147002002/02,0.11281036,376,3333.0272,-0.08228903
6,00df2b78-66c8-4009-89e0-0e14e15e4208,087067015/03,0.19108284,297,1554.2997,-0.38138286


### 5.1 Export

In [8]:
#build export path
exportPath <- paste(exportDirectory, 'populationDensitySmallArea2022', sep = '/')
exportPath <- paste(exportPath, '.csv', sep = '')

write.csv(smallAreaShpPop, exportPath, row.names = FALSE)


**END**